In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from pipeline.CrystalFinder import CrystalFinder

In [3]:
finder = CrystalFinder('../Models/Categorical-Model-12122020.h5')
preds = finder.get_predictions('../Minstrel-Screens/C2019015/1')

In [4]:
preds[preds["AI_predict"]=="Crystal"]

,filepath,image,AI_predict
6,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[74 73 68], [77 76 71], [82 80 75], [84 82 ...",Crystal
9,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[76 75 70], [74 73 68], [73 72 67], [74 73 ...",Crystal
18,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[157 157 143], [157 158 144], [157 158 143]...",Crystal
19,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[167 168 149], [170 172 151], [169 170 149]...",Crystal
27,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[95 93 81], [95 93 81], [94 90 81], [93 90 ...",Crystal
...,...,...,...
744,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[87 84 75], [89 86 77], [91 88 81], [91 88 ...",Crystal
752,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[93 90 85], [97 94 89], [93 90 85], [88 85 ...",Crystal
762,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[86 83 78], [85 82 77], [83 80 75], [83 80 ...",Crystal
765,../Minstrel-Screens/C2019015/1/RMM3_01_429_ROB...,"[[[[83 84 78], [85 86 80], [89 90 84], [91 92 ...",Crystal


In [33]:
scored = pd.read_csv('/home/grant/Documents/CSGID/CSGID_ImageRecognition/path-to-screens/minstrel-scored.csv')
scored = scored.append(list(scored.columns))
scored.dropna(axis=1, thresh=10, inplace=True)
scored.drop(columns=0, inplace=True)
scored.dropna(axis=0, how='any', inplace=True)
scored.columns = ["Timestamp", "Score_Number", "Score_Name", "Filepath"]

,Time,Score_number,Score_Name,Filepath
0,'2008-03-11 12:22:38.0','12','Bad Precipitate','images/C0000001/4/RMM3_01_429_ROBO_Test1_Clon...
1,'2008-05-15 15:00:01.0','13','Precipitate','images/C2000036/1/RMM3_01_429_ROBO_Test1_Clon...
2,'2008-05-15 15:00:31.0','13','Precipitate','images/C2000036/1/RMM3_01_429_ROBO_Test1_Clon...
3,'2008-05-15 15:02:00.0','33','Other','images/C2000038/1/RMM3_01_429_ROBO_Test1_Clon...
4,'2008-05-15 15:06:17.0','23','Crystals-bad form','images/C2000038/1/RMM3_01_429_ROBO_Test1_Clon...
...,...,...,...,...
54104,'2021-03-08 15:33:10.0','13','Precipitate','images/C2019257/5/RMM3_01_429_ROBO_Test1_Clon...
54105,'2021-03-08 15:33:40.0','13','Precipitate','images/C2019258/5/RMM3_01_429_ROBO_Test1_Clon...
54106,'2021-03-08 15:34:21.0','13','Precipitate','images/C2019259/5/RMM3_01_429_ROBO_Test1_Clon...
54107,'2021-03-08 15:34:56.0','13','Precipitate','images/C2019260/5/RMM3_01_429_ROBO_Test1_Clon...


In [37]:
minstrel_to_model = dict(zip(scored["Score_Name"].unique(),
                     ["Precipitate", "Precipitate", "Other", "Crystal", "Crystal",
                      "Other", "Precipitate", "Other", "Clear", "Crystal", "Other",
                      "Clear", "Other", "Crystal", "Crystal"]))

model_to_ai = {"Clear" : 0,
               "Crystal" : 1,
               "Precipitate" : 2,
               "Other" : 3,
               }

{"'Bad Precipitate'": 'Precipitate',
 "'Precipitate'": 'Precipitate',
 "'Other'": 'Other',
 "'Crystals-bad form'": 'Crystal',
 "'crystals - overnucleated'": 'Crystal',
 "'phase separation'": 'Other',
 "'Good precipitate'": 'Precipitate',
 "'Spherolites'": 'Other',
 "'Clear'": 'Clear',
 "'needles'": 'Crystal',
 "'Clear with stuff'": 'Other',
 "'Null experiment'": 'Clear',
 "'Small crystals'": 'Other',
 "'Crystals-harvestable'": 'Crystal'}

In [40]:
tf.config.list_physical_devices("GPU") # True/False

[]

In [ ]:
# Much of the code used here is based on: https://keras.io/examples/keras_recipes/tfrecord/
from utils.DataGenerator import get_dataset
import glob

train_list = glob.glob('../train-jpg/*')[0:2]
test_list = glob.glob('../test-jpg/*')[0:1]
valid_list = test_list

# Tuning and training params:
BATCH_SIZE = 1
IMAGE_SIZE = [299, 299, 3]

print("Train Files: ", len(train_list))
print("Test Files: ", len(test_list))

train_dataset = get_dataset(train_list, batch_size=BATCH_SIZE, im_size=IMAGE_SIZE)
valid_dataset = get_dataset(test_list, batch_size=BATCH_SIZE, im_size=IMAGE_SIZE)
test_dataset = get_dataset(test_list, labeled=False, batch_size=BATCH_SIZE, im_size=IMAGE_SIZE)

In [22]:
#import matplotlib.pyplot as plt
test_image, test_label = next(iter(valid_dataset))

In [18]:
test_pred = classifier.predict(test_image)
#plt.imshow(test_image[0].numpy()/255)

In [25]:
#Coded scores of individual images
np.argmax(test_pred, axis=1)

array([3, 0, 0, 0, 1, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3,
       1, 3, 3, 1, 0, 3, 3, 0, 3, 3, 0, 3, 2, 0, 0, 3, 0, 0, 0, 3, 3, 0,
       3, 0, 0, 3, 1, 3, 0, 3, 0, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 3, 0,
       3, 3, 0, 0, 0, 3, 3, 3, 0, 3, 1, 3, 0, 3, 2, 1, 3, 0, 3, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 0, 3])

In [23]:
test_image

<tf.Tensor: shape=(1, 299, 299, 3), dtype=float32, numpy=
array([[[[  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         ...,
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ]],

        [[  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         ...,
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ],
         [  0.      ,   0.      ,   0.      ]],

        [[102.02449 , 102.02449 , 102.02449 ],
         [ 99.071815,  99.071815,  99.071815],
         [ 97.00717 ,  97.00717 ,  97.00717 ],
         ...,
         [ 67.33158 ,  67.33158 ,  67.33158 ],
         [ 67.94145 ,  67.94145 ,  67.94145 ],
         [ 69.22516 ,  69.22516 ,  69.22516 ]],

        ...,

        [[110.616005, 110.616005, 

In [242]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def tfrecord_writer(filepath, label=5):
    image_string = open(filepath, 'rb').read()
    image_shape = tf.image.decode_jpeg(image_string).shape
    label = label
    feature = {
        "image/height": _int64_feature(image_shape[0]),  # image height in pixels
        "image/width": _int64_feature(image_shape[1]),  # image width in pixels
        "image/colorspace": _bytes_feature(bytes('RGB', encoding='utf8')),  # specifying the colorspace, always 'RGB'
        "image/channels": _int64_feature(3),  # specifying the number of channels, always 3
        "image/class/label": _int64_feature(label),  # specifying the index in a normalized classification layer
        "image/class/raw": _int64_feature(label),
        # specifying the index in the raw (original) classification layer
        "image/class/source": _int64_feature(5),
        # specifying the index of the source (creator of the image) - CSGID = 5
        "image/class/text": _bytes_feature(bytes(label)),
        # specifying the human-readable version of the normalized label
        "image/format": _bytes_feature(bytes("JPEG", encoding='utf8')),  # specifying the format, always 'JPEG'
        "image/filename": _bytes_feature(bytes(filepath, encoding='utf8' )),  # containing the basename of the image file
        "image/id": _bytes_feature(bytes(filepath, encoding='utf8')),  # specifying the unique id for the image
        "image/encoded": _bytes_feature(image_string),  # containing JPEG encoded image in RGB colorspace

    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [243]:
#Generate a dict of the well numbers and the image directories
image_labels = wells['filepath'].to_dict()
image_labels

{0: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_04_08_01_E0_99_025_001_RDI.jpg',
 1: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_05_08_01_E0_99_025_001_RDI.jpg',
 2: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_03_01_01_E0_99_025_001_RDI.jpg',
 3: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_03_03_01_E0_99_025_001_RDI.jpg',
 4: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_08_03_01_E0_99_025_001_RDI.jpg',
 5: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_02_08_01_E0_99_025_001_RDI.jpg',
 6: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_06_07_01_E0_99_025_001_RDI.jpg',
 7: '../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_10_08_01_E0_99_025_001_RDI.jpg',


In [244]:
tfrecord_batch = wells["filepath"].apply(lambda x: tfrecord_writer(x))

In [245]:
#where to save it:
record_file = '../test_images.tfrecords'

In [246]:
#Write all the serialized images to file:
with tf.io.TFRecordWriter(record_file) as writer:
    for label, filename in image_labels.items():
        print(filename)
        print(label)
        tf_example = tfrecord_writer(filename, label)
    writer.write(tf_example.SerializeToString())

../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_04_08_01_E0_99_025_001_RDI.jpg
0
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_05_08_01_E0_99_025_001_RDI.jpg
1
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_03_01_01_E0_99_025_001_RDI.jpg
2
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_03_03_01_E0_99_025_001_RDI.jpg
3
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_08_03_01_E0_99_025_001_RDI.jpg
4
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_02_08_01_E0_99_025_001_RDI.jpg
5
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_06_07_01_E0_99_025_001_RDI.jpg
6
../Minstrel-Screens/C2019025/1/RMM3_01_429_ROBO_Test1_Clon1_C2_0000C2019025_001_201030_10_08_01_E0_99_025_001_RDI.jpg
7
../Minstrel-Screens/C2019025/1/RMM3_01_4

In [249]:
pred_dataset = get_dataset(record_file, batch_size=BATCH_SIZE, im_size=IMAGE_SIZE)
#Let's make sure we can read our files:
raw_example = tf.data.TFRecordDataset(record_file)
parsed_example = raw_example.map(lambda x: tf.io.parse_example(x, feature_description))

#Read our training data from parsed_example into a real dataset:
imlist = [] #Vectorized images
label_list = [] #Machine-readable labels
human_label_list = [] #Human-readable labels

for n in parsed_example:
  this_image = tf.io.decode_jpeg(n['image/encoded'].values.numpy()[0]) #Get image data from TFR
  this_image = tf.cast(this_image, tf.int16) #Downsampling to int16
  this_image = tf.image.resize_with_crop_or_pad(  #Downsampling by cropping to drop
      this_image, 600,600
      )
  this_image = tf.image.resize(this_image, (IMAGE_SIZE[0], IMAGE_SIZE[1])) #Downsampling by resizing image
  this_label = n['image/class/label'].values.numpy()[0] #Get image class number
  this_human_label = n['image/class/text'].values.numpy()[0] #Get image class name

  this_image = (np.array(this_image) / 255)
  imlist.append(this_image)
  label_list.append(this_label)
  human_label_list.append(this_human_label)

imlist = np.array(imlist)
label_list = np.array(label_list)
human_label_list = np.array(human_label_list)

In [5]:
import datetime
str(datetime.datetime.now().date())

'2020-12-13'